In [1]:
# %load Apriori.py
#
import copy
import time
import sys
import pandas as pd
import numpy as np
import re

from collections import defaultdict
import csv
%load_ext memory_profiler

class Apriori(object):
    def __init__(self, minSupp, minConf):
        """ Parameters setting
        """
        self.minSupp = minSupp  # min support (used for mining frequent sets)
        self.minConf = minConf  # min confidence (used for mining association rules)

    def fit(self, data):
        """ Run the apriori algorithm, return the frequent *-term sets.
        """
        # Initialize some variables to hold the tmp result
        transListSet = data  # 取得transaction list
        itemSet = self.getOneItemSet(transListSet)  # get 1-item set
        itemCountDict = defaultdict(int)  # key=candiate k-item(k=1/2/...), value=count ,為不存在的 key 設定預設值int
        freqSet = dict()  # 儲存所有的 frequent *-items set

        self.transLength = len(transListSet)  # number of transactions
        self.itemSet = itemSet

        # Get the frequent 1-term set
        freqOneTermSet = self.getItemsWithMinSupp(transListSet, itemSet, itemCountDict, self.minSupp)  # L1

        # Main loop
        k = 1
        currFreqTermSet = freqOneTermSet  # 當前從L1開始計算
        while currFreqTermSet != set():  # 假設當前Lk不為空則迭代進行計算
            freqSet[k] = currFreqTermSet  # 儲存Lk items set
            k += 1  # k提升一階
            currCandiItemSet = self.getJoinedItemSet(currFreqTermSet, k)  # get new candiate k-terms set
            # get new frequent k-terms set
            currFreqTermSet = self.getItemsWithMinSupp(transListSet, currCandiItemSet,itemCountDict, self.minSupp)

        #
        self.itemCountDict = itemCountDict  # 儲存所有候選項以及出現的次數(不僅僅是頻繁項),用來計算置信度
        self.freqSet = freqSet  # dict: freqSet[k] indicate frequent k-term set Lk)
        return itemCountDict, freqSet

    def getSpecRules(self, rhs):
        """ Specify a right item, construct rules for it
        """
        if rhs not in self.itemSet: # rhs: L1中的某一個item
            print('Please input a term contain in the term-set !')
            return None

        rules = dict()
        for key, value in self.freqSet.items(): #從所有的frequent items set中尋找
            for itemSet in value:
                if rhs.issubset(itemSet) and len(itemSet) > 1: #假設rhs是屬於frequent itemset 中的子集，且該itemSet長度大於1
                    item_supp = self.getSupport(itemSet) #取得該itemSet的support值
                    itemSet = itemSet.difference(rhs) #取得該itemSet中的parent itemSet
                    conf = item_supp / self.getSupport(itemSet) # cond = sup(itemSet) / sup(parent itemSet)
                    if conf >= self.minConf:
                        rules[itemSet] = conf # 符合最小confidence值集加入規則
        return rules

    def getSupport(self, item):
        """ Get the support of item """
        # return self.itemCountDict[item] / self.transLength
        return self.itemCountDict[item]

    def getJoinedItemSet(self, termSet, k):
        """ Generate new k-terms candiate itemset"""
        return set([term1.union(term2) for term1 in termSet for term2 in termSet
                    if len(term1.union(term2)) == k])
        # 取Lk-1中的集合元素，兩兩互配形成Ck

    def getOneItemSet(self, transListSet):
        itemSet = set()
        for line in transListSet:
            for item in line:
                itemSet.add(frozenset([item]))  # 一一取出並納入集合中
        return itemSet

    def getItemsWithMinSupp(self, transListSet, itemSet, freqSet, minSupp):
        """ Get frequent item set using min support
        """
        itemSet_ = set()
        localSet_ = defaultdict(int)
        for item in itemSet:
            # 統計items在每筆transaction的出現次數(C1,...Cn)
            freqSet[item] += sum([1 for trans in transListSet if item.issubset(trans)])  # 納入frequent itemset集合
            localSet_[item] += sum([1 for trans in transListSet if item.issubset(trans)])  # 納入local frequent itemset集合

        # Only conserve frequent item-set
        n = len(transListSet)  # 取得Transaction Set長度以計算support
        for item, cnt in localSet_.items():
            # itemSet_.add(item) if float(cnt) / n >= minSupp else None
            itemSet_.add(item) if float(cnt) >= minSupp else None  # 統計符合minSupp的frequent items(L1,...Ln)

        return itemSet_


# if __name__ == '__main__':
    

def loadFromKaggle():

    bakery_data = pd.read_csv('BreadBasket_DMS.csv', encoding='utf-8')
    bakery_data['Date Time'] = bakery_data['Date'] + " " + bakery_data['Time']
    bakery_data = bakery_data.drop(['Date', 'Time'], axis=1)
    bakery_data = bakery_data.drop(['Date Time'], axis=1)
    bakery_data = bakery_data[~bakery_data['Item'].str.contains('NONE')]

    tdl = []
    for i in range(1, bakery_data.Transaction.count() + 1):
        tdf = bakery_data[bakery_data.Transaction == i]
        l = set()
        for j in range(0, tdf.Transaction.count()):
            l.add(tdf.Item.iloc[j])
        if len(l) > 0:
            tdl.append(list(l))
        else:
            tdl.append(None)

    col = ['items']
    TDB = pd.DataFrame({"items": tdl}, columns=col)
    TDB = TDB.dropna()
    return TDB['items'].tolist()


def loadFromIbm():
    # read data from Ibm
    with open('data', encoding='utf-8') as f:
        content = f.readlines()

    content = [x.strip() for x in content]

    Transaction = []  # to store transaction
    Frequent_items_value = {}  # to store all frequent item sets

    # to fill values in transaction from txt file
    Transaction = {}
    Transaction
    for i in range(0, len(content)):
        rowd = content[i].split(' ')
        rowd = [r for r in rowd if r != '']
        rowd = rowd[1:]
        if Transaction.get(rowd[0], None) == None:
            Transaction[rowd[0]] = [rowd[1]]
        else:
            Transaction[rowd[0]].append(rowd[1])
    # print(type(list(Transaction.values())))
    return list(Transaction.values())


# cd D:\WorkSpace\PythonWorkSpace\Apriori
# D:
# python Apriori.py ibm 2 0.6
def test1():
    fn = 'BreadBasket_DMS.csv'  # BreadBasket_DMS.csv
    minSup = 16  # 3
    minConf = 0.6  # 0.6
    print('fileName = {} ,minSup= {} , minConf={}'.format(fn, minSup, minConf))

    starttime = time.time()
#     if fn == 'kaggle':
#     dataSet = loadFromKaggle()
#     elif fn == 'ibm':
#         dataSet = loadFromIbm()
    dataSet = loadFromIbm()
    # Run
    objApriori = Apriori(minSup, minConf)
    itemCountDict, freqSet = objApriori.fit(dataSet)
    #     print(itemCountDict)
    endtime = time.time()
    print("\nTime Taken is: {0:.2f}ms \n".format((endtime - starttime)))
    #   print
    for key, value in freqSet.items():
        print('{}-Itemsets:{}'.format(key, len(value)))
        print('-' * 20)
        for itemset in value:
            print("Items :{} , Support:{} ".format(list(itemset), itemCountDict[itemset]))
        print()

    # Return rules with regard of `rhs`
    print()
    print('List All Rules:')
    print()
    L1 = set(freqSet[1])
    for rhs in L1:
        rules = objApriori.getSpecRules(rhs)
        if len(rules) > 0:
            #             print('-'*20)
            #             print('rules refer to {}'.format(list(rhs)))
            for key, value in rules.items():
                print('Rule : {} -> {}, confidence = {}'.format(list(key), list(rhs), value))
#     %memit test()
# %mprun -T mprof0 -f test test()


In [2]:
%memit test1

peak memory: 87.71 MiB, increment: 0.06 MiB


In [3]:
import itertools
import time
import sys
import copy

# function to get frequent one itemset
def frequent_one_item(Transaction, min_support):
	candidate1 = {}

	for i in range(0, len(Transaction)):  # 計數C1 item中的出現次數
		for j in range(0, len(Transaction[i])):
			if Transaction[i][j] not in candidate1:
				candidate1[Transaction[i][j]] = 1
			else:
				candidate1[Transaction[i][j]] += 1

	frequentitem1 = []  # to get frequent 1 itemsets with minimum support count
	for value in candidate1:
		if candidate1[value] >= min_support:
			frequentitem1 = frequentitem1 + [[value]]
			Frequent_items_value[tuple(value)] = candidate1[value]

	return frequentitem1


def printL1(L1):
	print("1-itemsets :{}".format(len(L1.items())))
	for key, value in L1.items():
		key = str(key).replace('(', '[').replace(')', ']')
		print("Items: {}, Support :{}".format(key, value))


def print_L(L_value):
	for i, L in enumerate(L_value):
		if i < 2:
			pass
		elif len(L) == 0:
			pass
		else:
			print()
			print("{}-itemsets :{}".format((i), len(L)))
			[print('Items: {} , Support: '.format(itemSet)) for itemSet in L]


# class of Hash node
class Hash_node:
	def __init__(self):
		self.children = {}  # pointer to its children
		self.Is_Leaf = True  # to know the status whether current node is leaf or not
		self.bucket = {}  # contains itemsets in bucket


# class of constructing and getting hashtree
class HashTree:
	# class constructor
	def __init__(self, max_leaf_count, max_child_count):
		self.root = Hash_node()
		self.max_leaf_count = max_leaf_count
		self.max_child_count = max_child_count
		self.frequent_itemsets = []

	# function to recursive insertion to make hashtree
	def recursively_insert(self, node, itemset, index, count):
		if index == len(itemset):  # 如果當前插入索引已滿
			if itemset in node.bucket:  # bucket : HashTree的緩衝儲存區
				node.bucket[itemset] += count  # 若ck itemset已存在bucket中，則在bucket中計數值加一次count
			else:
				node.bucket[itemset] = count  # 若ck itemset已存在bucket中，則在bucket中計數值等於當前count
			return

		if node.Is_Leaf:  # if node is leaf
			if itemset in node.bucket:
				node.bucket[itemset] += count  # 若ck itemset已存在bucket中，則在bucket中計數值加一次count
			else:
				node.bucket[itemset] = count  # 若ck itemset已存在bucket中，則在bucket中計數值等於當前count
			if len(node.bucket) == self.max_leaf_count:  # 如果bucket可容納的數量已經達到達到leaf node的乘載最大數量
				for old_itemset, old_count in node.bucket.items():  # 取出bucket中舊有的itemset和count

					hash_key = self.hash_function(old_itemset[index])  # 將其 hashing 至另外一個 index
					if hash_key not in node.children:  # 如果Node底下並沒有以hash_key為首的children HT
						node.children[hash_key] = Hash_node()  # 在Node底下做一棵以hash_key為首的children HT
					self.recursively_insert(node.children[hash_key], old_itemset, index + 1, old_count)
				# 將bucket中舊有的itemset遞迴插入至children HT
				# since no more requirement of this bucket
				del node.bucket  # 將緩衝儲存區的內容清除
				node.Is_Leaf = False  # 該節點成為Non-Leaf node
		else:  # if node is not leaf
			hash_key = self.hash_function(itemset[index])
			if hash_key not in node.children:  # 如果Non-Leaf node底下並沒有以hash_key為首的children HT
				node.children[hash_key] = Hash_node()  # 在Node底下做一棵以hash_key為首的children HT
			self.recursively_insert(node.children[hash_key], itemset, index + 1, count)
		# 將itemset遞迴插入至指定的Hash children HT

	def insert(self, itemset):
		itemset = tuple(itemset)  # 將ck中的itemset轉化成dict結構
		self.recursively_insert(self.root, itemset, 0, 0)  # 進行遞迴插入

	# to add support to candidate itemsets. Transverse the Tree and find the bucket in which this itemset is present.
	def add_support(self, itemset):
		Transverse_HNode = self.root
		itemset = tuple(itemset)
		index = 0
		while True:
			if Transverse_HNode.Is_Leaf:
				if itemset in Transverse_HNode.bucket:  # found the itemset in this bucket
					Transverse_HNode.bucket[itemset] += 1  # increment the count of this itemset.
				break
			hash_key = self.hash_function(itemset[index])
			if hash_key in Transverse_HNode.children:
				Transverse_HNode = Transverse_HNode.children[hash_key]
			else:
				break
			index += 1

	# to transverse the hashtree to get frequent itemsets with minimum support count
	def get_frequent_itemsets(self, node, support_count, frequent_itemsets):
		if node.Is_Leaf:
			for key, value in node.bucket.items():
				if value >= support_count:  # if it satisfies the condition
					frequent_itemsets.append(list(key))  # then add it to frequent itemsets.
					Frequent_items_value[key] = value
			return

		for child in node.children.values():
			self.get_frequent_itemsets(child, support_count, frequent_itemsets)

	# hash function for making HashTree
	def hash_function(self, val):
		return int(val) % self.max_child_count


# To generate hash tree from candidate itemsets
def generate_hash_tree(candidate_itemsets, max_leaf_count, max_child_count):
	htree = HashTree(max_child_count, max_leaf_count)  # create instance of HashTree
	for itemset in candidate_itemsets:
		htree.insert(itemset)  # to insert itemset into Hashtree
	return htree


# to generate subsets of itemsets of size k
def generate_k_subsets(dataset, length):
	subsets = []
	for itemset in dataset:
		subsets.extend(map(list, itertools.combinations(itemset, length)))
	return subsets


def subset_generation(ck_data, l):
	# itertools.combinations(iterable, r)
	# 創建一個迭代器，返回iterable中所有長度為r的子序列，返回的子序列中的項按輸入iterable中的順序排序（不帶重複）
	return map(list, set(itertools.combinations(ck_data, l)))


# apriori generate function to generate ck
def apriori_generate(Lk, k):
	ck = []
	# join step
	lenlk = len(Lk)
	for i in range(lenlk):  # 從Lk中的itemsets取出元素相互混種
		for j in range(i + 1, lenlk):
			L1 = list(Lk[i])[:k - 2]
			L2 = list(Lk[j])[:k - 2]
			if L1 == L2:
				ck.append(sorted(list(set(Lk[i]) | set(Lk[j]))))  # 將新混種出來的itemsets放入Ck

			# prune step
	final_ck = []
	for candidate in ck:
		all_subsets = list(subset_generation(set(candidate), k - 1))  # 產生Ck-1的Subsets
		found = True

		# -- 改良--#
		for subset in all_subsets:
			subset = list(sorted(subset))
			if (subset not in Lk) and (subset in ck):
				ck.remove(subset)

			# -- 舊有--#
			# for i in range(len(all_subsets)):
			#     value = list(sorted(all_subsets[i]))
			#     if value not in Lk:
			#         found = False
			# if found == True:
			#     final_ck.append(candidate)

	return ck, final_ck


def generateL(ck, min_support):
	support_ck = {}
	for val in Transaction1:
		for val1 in ck:
			value = set(val)
			value1 = set(val1)

			if value1.issubset(value):
				if tuple(val1) not in support_ck:
					support_ck[tuple(val1)] = 1
				else:
					support_ck[tuple(val1)] += 1
	frequent_item = []
	for item_set in support_ck:
		if support_ck[item_set] >= min_support:
			frequent_item.append(sorted(list(item_set)))
			Frequent_items_value[item_set] = support_ck[item_set]

	return frequent_item


# main apriori algorithm function
def apriori(L1, min_support, max_leaf_count=3, max_child_count=5):
	k = 2;
	L = []
	L.append(0)
	L.append(L1)

	start = time.time()
	while (len(L[k - 1]) > 0):  # 假如Lk-1裡面還有itemsets就繼續做
		ck, final_ck = apriori_generate(L[k - 1], k)  # to generate candidate itemsets
		h_tree = generate_hash_tree(ck, max_leaf_count, max_child_count)  # to generate hashtree
		k_subsets = generate_k_subsets(Transaction1, k)  # to generate subsets of each transaction
		for subset in k_subsets:
			h_tree.add_support(subset)  # hashtree中的每個node加上support
		lk = []
		h_tree.get_frequent_itemsets(h_tree.root, min_support, lk)  # 從hashtree中取出每個 frequent itemsets
		L.append(lk)
		k = k + 1  # Ck,Lk提升一階
	end = time.time()
	return L, (end - start)


def generateL1(L1, Transaction_len):
	print("1-itemsets :{}".format(len(L1.items())))
	cpL1 = copy.deepcopy(L1)

	for key, value in cpL1.items():
		# key = str(key).replace('(', '[').replace(')', ']')
		# key = int(key)
		item = 0
		for i in range(0,len(key)):
			item = item + int(key[i])
			item*=10
			L1[item] = value
		del L1[key]
		print("Items: {} , Support :{}".format([item], value))


def generateLn(L_value):
	for i, L in enumerate(L_value):
		if i < 2:
			pass
		elif len(L) == 0:
			pass
		else:
			print()
			print("{}-itemsets :{}".format((i), len(L)))
			[print('Items: {} , Support:{}'.format(itemSet,Frequent_items_value[tuple(itemSet)])) for itemSet in L]


# if __name__ == '__main__':
	import itertools
	import time


def loadFromKaggle():

    bakery_data = pd.read_csv('BreadBasket_DMS.csv', encoding='utf-8')
    bakery_data['Date Time'] = bakery_data['Date'] + " " + bakery_data['Time']
    bakery_data = bakery_data.drop(['Date', 'Time'], axis=1)
    bakery_data = bakery_data.drop(['Date Time'], axis=1)
    bakery_data = bakery_data[~bakery_data['Item'].str.contains('NONE')]

    tdl = []
    for i in range(1, bakery_data.Transaction.count() + 1):
        tdf = bakery_data[bakery_data.Transaction == i]
        l = set()
        for j in range(0, tdf.Transaction.count()):
            l.add(tdf.Item.iloc[j])
        if len(l) > 0:
            tdl.append(list(l))
        else:
            tdl.append(None)

    col = ['items']
    TDB = pd.DataFrame({"items": tdl}, columns=col)
    TDB = TDB.dropna()
    return TDB['items'].tolist()
def loadFromIbm():
    # read data from Ibm
    with open('data', encoding='utf-8') as f:
        content = f.readlines()

    content = [x.strip() for x in content]

    Transaction = []  # to store transaction
    Frequent_items_value = {}  # to store all frequent item sets

    # to fill values in transaction from txt file
    Transaction = {}
    Transaction
    for i in range(0, len(content)):
        rowd = content[i].split(' ')
        rowd = [r for r in rowd if r != '']
        rowd = rowd[1:]
        if Transaction.get(rowd[0], None) == None:
            Transaction[rowd[0]] = [rowd[1]]
        else:
            Transaction[rowd[0]].append(rowd[1])
    # print(type(list(Transaction.values())))
    return list(Transaction.values())



def test2():
    # cd D:\WorkSpace\PythonWorkSpace\Apriori
    # D:
    # python Apriori_HT.py ibm 10 3 5
#     fn = str(sys.argv[1])
    minSup = 16  # 3
    max_leaf_count = 3  # 3
    max_child_count = 5  # 5
    print('fileName = {} ,minSup= {} , max_leaf_count={} , max_child_count={} '.format(fn, minSup, max_leaf_count,max_child_count))

    starttime = time.time()

    Frequent_items_value = {}  # to store all frequent item sets

#     if fn == 'ibm':
    Transaction = loadFromIbm()

    Transaction_len = len(Transaction)

    print("All frequent itemsets with their support count:")
    L1 = frequent_one_item(Transaction, minSup)
    generateL1(Frequent_items_value, Transaction_len)

    # to remove infrequent 1 itemsets from transaction
    Transaction1 = []
    for i in range(0, len(Transaction)):
        list_val = []
        for j in range(0, len(Transaction[i])):
            if [Transaction[i][j]] in L1:
                list_val.append(Transaction[i][j])
        Transaction1.append(list_val)

    L_value, time_taken = apriori(L1, minSup, max_leaf_count, max_child_count)
    generateLn(L_value)

    print("\nTime Taken is: {0:.3f} ms\n".format(time_taken))


In [4]:
# %memit test1
%memit test2

peak memory: 89.52 MiB, increment: 0.01 MiB


In [5]:
def transfer2FrozenDataSet(dataSet):
    frozenDataSet = {}
    for elem in dataSet:
        frozenDataSet[frozenset(elem)] = 1
    return frozenDataSet


class TreeNode:
    def __init__(self, nodeName, count, nodeParent):
        self.nodeName = nodeName
        self.count = count
        self.nodeParent = nodeParent
        self.nextSimilarItem = None  # 指向下一個相同元素的指針nextSimilarItem
        self.children = {}

    def updateC(self, count):
        self.count += count


def createFPTree(frozenDataSet, minSupport):
    # scan dataset at the first time, filter out items which are less than minSupport
    frequenctNodeTable = {}  # 紀錄每個item的出現數目,還有其parent Node
    for transactions in frozenDataSet:  # 依序取出每筆transactions
        for item in transactions:  # 在取出每項item
            item = str(item)
            frequenctNodeTable[item] = frequenctNodeTable.get(item, 0) + frozenDataSet[transactions]  # 取出每項的count+1


            # count = frequenctNodeTable.get(item, default = 0) , 1 = frozenDataSet[transactions]
    frequenctNodeTable = {k: v for k, v in frequenctNodeTable.items() if v >= minSupport}  # 濾除掉不滿足minSupport的item
    frequentItemSet = set(frequenctNodeTable.keys())  # 紀錄滿足minSupport items的集合

    if len(frequentItemSet) == 0: return None, None

    for k in frequenctNodeTable:
        # 將frequenctNodeTable中的結構(item,count) -> (item,[count,ItemNode(default=None)])
        frequenctNodeTable[k] = [frequenctNodeTable[k], None]  # (初始化所有非root Node的所有資訊)

    fptree = TreeNode("null", 1, None)  # 初始化FP-Tree Root Node
    # scan dataset at the second time, filter out items for each record
    for transactions, count in frozenDataSet.items():  # 再次scanfrozenDataSet中的所有items
        frequentItemsInRecord = {}  # 紀錄transaction中的frequent items
        for item in transactions:
            if item in frequentItemSet:  # 利用frequentItemSet過濾出每筆transaction中的常見items
                frequentItemsInRecord[item] = frequenctNodeTable[item][0]
        if len(frequentItemsInRecord) > 0:
            # 將transaction中的items依照support大小作排序(大的在前)，形成一條FP path
            orderedFrequentItems = [v[0] for v in sorted(frequentItemsInRecord.items(), key=lambda v: v[1],
                                                         reverse=True)]  # sort by v[1] (support)降序
            updateFPTree(fptree, orderedFrequentItems, frequenctNodeTable, count)

    return fptree, frequenctNodeTable


def updateFPTree(fptree, orderedFrequentItems, frequenctNodeTable, count):
    # handle the first item
    firstOrderItem = orderedFrequentItems[0]  # scan FP-path 的第1個item
    if firstOrderItem in fptree.children:  # 如果firstOrderItem是fptree 節點的子節點
        fptree.children[firstOrderItem].updateC(count)  # 增加該子節點的count數
    else:
        fptree.children[firstOrderItem] = TreeNode(firstOrderItem, count,
                                                   fptree)  # 否則，需要新創建一個TreeNode 節點，然後將其賦給fptree 節點的子節點
        # update frequenctNodeTable
        if frequenctNodeTable[firstOrderItem][1] == None:  # 如果frequenctNodeTable裡記錄的firstOrderItem Node為None
            frequenctNodeTable[firstOrderItem][1] = fptree.children[
                firstOrderItem]  # 則更新frequenctNodeTable裡的firstOrderItem Node為自己本身
        else:
            updateFrequenctNodeTable(frequenctNodeTable[firstOrderItem][1],
                                     fptree.children[firstOrderItem])  # 更新 firstOrderItem 下一個相似元素指標
    # handle other items except the first item
    if (len(orderedFrequentItems) > 1):  # 假如 FP-path 的後面還有其他item
        updateFPTree(fptree.children[firstOrderItem], orderedFrequentItems[1:], frequenctNodeTable, count)
        # 以FP-path上的firstOrderItem node為新的root Node，處理後續的orderedFrequentItems


def updateFrequenctNodeTable(headFrequentNode, targetNode):
    while (headFrequentNode.nextSimilarItem != None):
        headFrequentNode = headFrequentNode.nextSimilarItem  # 不斷尋找相似元素指標為空的headFrequentNode
    headFrequentNode.nextSimilarItem = targetNode  # 將headFrequentNode 與 targetNode 以相似元素指標連接


def mining_FPTree(frequenctNodeTable, suffix, frequentPatterns, minSupport):
    # for each item in frequenctNodeTable, find conditional suffix path, create conditional fptree, then iterate until there is only one element in conditional fptree
    if (frequenctNodeTable == None): return
    frequenctItems = [v[0] for v in sorted(frequenctNodeTable.items(),
                                           key=lambda v: v[1][0])]  # 將frequenctNodeTable以升序排序，從low frequent開始mining
    if (len(frequenctItems) == 0): return

    for frequenctItem in frequenctItems:
        newSuffix = suffix.copy()  # 只拷貝父級目錄，子目錄不拷貝
        newSuffix.add(frequenctItem)  # 加入一個擁有最低出現次數的Suffix item作為newSuffix
        support = frequenctNodeTable[frequenctItem][0]  # 逐一取得Suffix item的support
        frequentPatterns[frozenset(newSuffix)] = support  # 逐一取得Suffix item的support，並將其一一加入到frequentPatterns

        suffixPath = getSuffixPath(frequenctNodeTable, frequenctItem)  # Find suffix patterns to the frequenctItem
        if (suffixPath != {}):
            conditionalFPtree, conditionalFNodeTable = createFPTree(suffixPath, minSupport)
            # 根據suffixPath建立conditionalFPtree 和 conditional frequenctNode Table
            if conditionalFNodeTable != None:  # 假如 conditional frequenctNode不為空
                mining_FPTree(conditionalFNodeTable, newSuffix, frequentPatterns,
                              minSupport)  # 繼續挖掘剩餘的conditionalFPtree


def getSuffixPath(frequenctNodeTable, frequenctItem):
    suffixPath = {}
    beginNode = frequenctNodeTable[frequenctItem][1]  # 指向當前frequenctItem 的 Tree Node
    suffixs = ascendNodeList(beginNode)  # suffixs紀錄由beginNode往上層尋找的ascendTree (不包含beginNode)
    if ((suffixs != [])):
        suffixPath[
            frozenset(suffixs)] = beginNode.count  # 建立一條由suffix Node起始但不包含suffix Node的完整suffixPath，數量為beginNode的出現次數

    while (beginNode.nextSimilarItem != None):
        beginNode = beginNode.nextSimilarItem
        suffixs = ascendNodeList(beginNode)
        if (suffixs != []):
            suffixPath[frozenset(suffixs)] = beginNode.count

    return suffixPath  # 回傳suffixPath


def ascendNodeList(treeNode):
    suffixs = []
    while ((treeNode.nodeParent != None) and (treeNode.nodeParent.nodeName != 'null')):
        treeNode = treeNode.nodeParent
        suffixs.append(treeNode.nodeName)
    return suffixs


def rulesGenerator(frequentPatterns, minConf, rules):
    for frequentset in frequentPatterns:
        if (len(frequentset) > 1):  # 依序取得所有長度大於1的frequentset
            getRules(frequentset, frequentset, rules, frequentPatterns, minConf)
            # 透過frequentset自己本身去產生規則並輸出到rules內


def removeStr(set, str):
    tempSet = []
    for elem in set:
        if (elem != str):
            tempSet.append(elem)  # 將不等於frequentElem的項目篩選出來，並加入到tempSet中
    tempFrozenSet = frozenset(tempSet)
    return tempFrozenSet


def getRules(frequentset, currentset, rules, frequentPatterns, minConf):  # 由大至小拆解各集合，以取得規則
    for frequentElem in currentset:
        subSet = removeStr(currentset, frequentElem)  # subSet為不包含frequentElem的子集合
        # print(currentset)
        # print(frequentElem)
        confidence = frequentPatterns.get(frequentset) / frequentPatterns.get(subSet, 9999)
        # confidence = sup(currentset) / sup(subSet)
        if (confidence >= minConf):
            flag = False
            for rule in rules:
                # rule[0] : 推演規則  # rule[1] : 衍伸規則
                if (rule[0] == subSet and rule[1] == frequentset - subSet):
                    flag = True
            if (flag == False):
                rules.append((subSet, frequentset - subSet, confidence))
                # rules.append (推演規則 -------> 衍伸規則 , confidence)

            if (len(subSet) >= 2):  # 如果subSet中的frequent items數大於2
                getRules(frequentset, subSet, rules, frequentPatterns, minConf)
                # 以subSet為目標繼續挖掘其他的規則


def getpatterns(pattern):
    maxVal = max(pattern.items())[1]
    dictpattern = {}
    for i in range(1, maxVal + 10):
        dictpattern[str(i)] = []

    for key, value in pattern.items():
        dictpattern[str(len(key))].append([key, value])

    for i in range(1, maxVal + 10):
        patterns = dictpattern[str(i)]
        if len(patterns) > 0:
            print("{}-items set :{}".format(str(i),len(patterns)))
            [print("items:{} , support:{} ".format(set(item[0]), item[1])) for item in patterns]
            print()


# if __name__ == '__main__':
def loadFromKaggle():

    bakery_data = pd.read_csv('BreadBasket_DMS.csv', encoding='utf-8')
    bakery_data['Date Time'] = bakery_data['Date'] + " " + bakery_data['Time']
    bakery_data = bakery_data.drop(['Date', 'Time'], axis=1)
    bakery_data = bakery_data.drop(['Date Time'], axis=1)
    bakery_data = bakery_data[~bakery_data['Item'].str.contains('NONE')]

    tdl = []
    for i in range(1, bakery_data.Transaction.count() + 1):
        tdf = bakery_data[bakery_data.Transaction == i]
        l = set()
        for j in range(0, tdf.Transaction.count()):
            l.add(tdf.Item.iloc[j])
        if len(l) > 0:
            tdl.append(list(l))
        else:
            tdl.append(None)

    col = ['items']
    TDB = pd.DataFrame({"items": tdl}, columns=col)
    TDB = TDB.dropna()
    return TDB['items'].tolist()


def loadFromIbm():
    # read data from Ibm
    with open('data', encoding='utf-8') as f:
        content = f.readlines()

    content = [x.strip() for x in content]

    Transaction = []  # to store transaction
    Frequent_items_value = {}  # to store all frequent item sets

    # to fill values in transaction from txt file
    Transaction = {}
    Transaction
    for i in range(0, len(content)):
        rowd = content[i].split(' ')
        rowd = [r for r in rowd if r != '']
        rowd = rowd[1:]
        if Transaction.get(rowd[0], None) == None:
            Transaction[rowd[0]] = [rowd[1]]
        else:
            Transaction[rowd[0]].append(rowd[1])
    # print(type(list(Transaction.values())))
    return list(Transaction.values())

def test3():
    # cd D:\WorkSpace\PythonWorkSpace\Apriori
    # D:
    # python FP-Growth.py ibm 10 0.6
#     fn = str(sys.argv[1])  # BreadBasket_DMS.csv
    minSup = 30  # 3
    minConf = 0.6  # 0.6
    print('fileName = {} ,minSup= {} , minConf={}'.format(fn, minSup, minConf))

    starttime = time.time()
    # if fn == 'kaggle':
    dataSet = loadFromKaggle()
    # elif fn == 'ibm':
#     dataSet = loadFromIbm()

    frozenDataSet = transfer2FrozenDataSet(dataSet)  # 為transaction中所有曾經出現的item建立Node，並初始化support為1
    minSupport = 3
    fptree, frequenctNodeTable = createFPTree(frozenDataSet, minSup)
    frequentPatterns = {}

    suffix = set([])
    mining_FPTree(frequenctNodeTable, suffix, frequentPatterns, minSup)

    rules = []
    endtime = time.time()
    print("fptree:")
    print("\nTime Taken is: {0:.2f}ms \n".format((endtime - starttime)))

    print("frequent patterns:")
    getpatterns(frequentPatterns)

    print("association rules:")
    rulesGenerator(frequentPatterns, minConf, rules)
    rules = [rule for rule in rules if rule != None]
    [print('Rules:{}-->{}, confidence:{}'.format(set(r[0]), set(r[1]), r[2])) for r in rules]


In [6]:
%memit test3

peak memory: 89.35 MiB, increment: 0.02 MiB
